In [ ]:
# default_exp crawler

# Crawler
> This repository aims to explore the catalog available at wine.com.br, do some exploratory analysis in it and 
create initially a toy recommendation engine / wine classifier and pricing tool. 

In [ ]:
#hide
from nbdev.showdoc import *
import scrapy
from fire import Fire
from bs4 import BeautifulSoup as soup
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors.lxmlhtml import LxmlLinkExtractor

prefix = 'https://wine.com.br'
url_short = 'https://www.wine.com.br/browse.ep?cID=100851&exibirEsgotados=true&listagem=horizontal&sorter=featuredProducts-desc&filters=cVINHOS'

In [ ]:
CATALOG = '//article/div[2]'
NEXT = '/html/body/div[6]/div/div[2]/div[2]/div/div[4]/div'

In [ ]:
link_extract =   LxmlLinkExtractor(restrict_text='Próxima >>')    

In [ ]:
# Create the Spider class
class WineSpider(scrapy.Spider):
    name = "winespider"
    def start_requests(self):
        yield scrapy.Request(url = url_short, callback = self.parse_page)
    
    def parse_page(self, response):
        wine_list = response.xpath(CATALOG)
        for block in wine_list:
            yield { 'wine': block.css('div > a::attr("title")').get(),
                    'link': prefix + block.css('div > a::attr("href")').get()
                  }
        
        #next_page = response.xpath(f"{NEXT}//a[./text()='Próxima >>']").get()
        next_page = response.xpath("//a[contains(text(), 'Próxima')]/href").get()
        
        
        if next_page is not None:
            yield response.follow(next_page, self.parse_page)
        
    # Second parsing method
    def parse_pages(self, response):
        crs_title = response.xpath('//h1[contains(@class,"title")]/text()')
        crs_title_ext = crs_title.extract_first().strip()
        ch_titles = response.css('p.course__description::text')
        ch_titles_ext = [t.strip() for t in ch_titles.extract()]
        dc_dict[ crs_title_ext ] = ch_titles_ext

In [ ]:
# Initialize the dictionary **outside** of the Spider class
dc_dict = dict()


def crawl():
    # Run the Spider
    process = CrawlerProcess()
    process.crawl(WineSpider)
    process.start()

if __name__ == '__main__':
    Fire(crawl)

2020-02-16 17:11:57 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-02-16 17:11:57 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug 13 2019, 20:35:49) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.10.0-38-generic-x86_64-with-debian-stretch-sid
2020-02-16 17:11:57 [scrapy.crawler] INFO: Overridden settings: {}
2020-02-16 17:11:57 [scrapy.extensions.telnet] INFO: Telnet Password: 6d6c61681a4ac7bc
2020-02-16 17:11:57 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2020-02-16 17:11:57 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.Downl

FireExit: 2

2020-02-16 17:12:01 [py.warnings] WARNING: /home/ronaldo/.local/share/virtualenvs/wino-rEbptTiD/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)

